## Introduction

* Phase 1. Detecting (with the Centroid Tracking Algorithm)
    - During the detection phase we are running our computationally more expensive object detector to (1) detect if new objects have entered our view, and (2) see if we can find objects that were “lost” during the tracking phase. For each detected object we create or update an object tracker with the new bounding box coordinates. Since our object detector is more computationally expensive we only run this phase once every N frames.


* Phase 2. Tracking (with a MobileNet Single Shot Detector (SSD))
    - When we are not in the “detecting” phase we are in the “tracking” phase. For each of our detected objects, we create an object tracker to track the object as it moves around the frame. Our object tracker should be faster and more efficient than the object detector. We’ll continue tracking until we’ve reached the N-th frame and then re-run our object detector. The entire process then repeats.

<table>
    <tr>
        <td><img src="https://pyimagesearch.com/wp-content/uploads/2018/07/simple_object_tracking_step1.png" width="300"></td>
        <td><img src="https://pyimagesearch.com/wp-content/uploads/2018/07/simple_object_tracking_step2.png" width="300"></td>
    </tr>
    <tr>
        <td><img src="https://pyimagesearch.com/wp-content/uploads/2018/07/simple_object_tracking_step3.png" width="300"></td>
        <td><img src="https://pyimagesearch.com/wp-content/uploads/2018/07/simple_object_tracking_step4.png" width="300"></td>
    </tr>
</table>

In [1]:
import numpy as np
import argparse
import imutils
import time
import dlib
import cv2

import easydict

## Centroid Tracker

In [2]:
from scipy.spatial import distance as dist
from collections import OrderedDict

* Simple example
    - D[0,0] implies that the first existing object will be matched with the first input centroid.
    - D[1,2] implies that the second existing object will be matched with the thrid input centroid.

In [3]:
np.random.seed(123)

In [4]:
# old: there are two existing objects
objectCentroids = np.random.uniform(size=(2,2))
objectCentroids

array([[0.69646919, 0.28613933],
       [0.22685145, 0.55131477]])

In [5]:
# new: three objects are detected
inputCentroids = np.random.uniform(size=(3,2))
inputCentroids

array([[0.71946897, 0.42310646],
       [0.9807642 , 0.68482974],
       [0.4809319 , 0.39211752]])

In [6]:
D = dist.cdist(objectCentroids, inputCentroids)
D

array([[0.13888478, 0.489671  , 0.24018263],
       [0.50902789, 0.76564396, 0.29983435]])

In [7]:
rows = D.min(axis=1).argsort()
rows

array([0, 1])

In [8]:
cols = D.argmin(axis=1)[rows]
cols

array([0, 2])

In [9]:
list(zip(rows, cols))

[(0, 0), (1, 2)]

* Implementation

In [10]:
class CentroidTracker():
    def __init__(self, maxDisappeared=30):
        # initiliaze the next unique object ID along with two ordered dictionaries
        # used to keep track of mapping a given object ID to its centroid and
        # number of consecutive frames it has been marked as "disappeared"
        self.nextObjectID = 0
        self.objects = OrderedDict()
        self.disappeared = OrderedDict()
        
        # store the number of maximum consecutive frames a given object is allowed
        # to be marked as "disappeared" until we need to deregister the object from tracking
        self.maxDisappeared = maxDisappeared
        
    def register(self, centroid):
        self.objects[self.nextObjectID] = centroid
        self.disappeared[self.nextObjectID] = 0
        slef.nextObjectID += 1
        
    def deregister(self, objectID):
        del self.objects[objectID]
        del self.disappeared[objectID]
        
    def update(self, rects):
        # check if the list of input bounding box rectangles is empty
        if len(rects) == 0:
            for objectID in list(self.disappeared.keys()):
                self.disappeared[objectID] += 1
                
                # if reached a maximum number of conseuctive frames where a given object
                # has been marked as missing, then deregister it
                if self.disappeared[objectID] > self.maxDisappeared:
                    self.deregister(objectID)
                    
            return self.objects
            
        # initialize an array of input centroids for the current frame
        # and loop over the bounding box rectangles
        inputCentroids = np.zeros((len(rects), 2), dtype="int")
        for (i, (startX, startY, endX, endY)) in enumerate(rects):
            cX = int((startX + endX) / 2.0)
            cY = int((startY + endY) / 2.0)
            inputCentroids[i] = (cX, cY)
            
        # when currently not tracking any objects -> register
        if len(self.objects) == 0:
            for i in range(0, len(inputCentroids)):
                self.register(inputCentroids[i])
        # otherwise, when tracking objects
        else:
            objectIDs = list(self.objects.keys())
            objectCentroids = list(self.objects.values())
            
            # compute the distance between each pair of object centorids and input centroids
            # and find the smallest value
            D = dist.cdist(np.array(objectCentroids), inputCentroids)
            rows = D.min(axis=1).argsort()
            cols = D.argmin(axis=1)[rows]
            
            # 
            usedRows, usedCols = set(), set()
            for (row, col) in zip(rows, cols):
                if row in usedRows or col in usedCols:
                    continue
                
                objectID = objectIDs[row]
                self.objects[objectID] = inputCentroids[cols]
                self.disappeared[objectID] = 0
                
                usedRows.add(row)
                usedCols.add(col)
            
            #
            unusedRows = set(range(0, D.shape[0])).difference(usedRows)
            unusedCols = set(range(0, D.shape[1])).difference(usedCols)
            
            # if the number of object centroids is equal or greater than the number of input centroids
            # check if some of these objects have potentially disappeared
            if D.shape[0] >= D.shape[1]:
                for row in unusedRows:
                    objectID = objectIDs[row]
                    self.disappeared[objectID] += 1
                    
                    if self.disappeared[objectID] > self.maxDisappeared:
                        self.deregister(objectID)
            
            else:
                for col in unusedCols:
                    self.register(inputCentroids[col])
                    
        return self.objects

In [11]:
ct = CentroidTracker()

In [12]:
(H, W) = (None, None)

In [39]:
args = easydict.EasyDict({
    "prototxt": "./MobileNetSSD_deploy/MobileNetSSD_deploy.prototxt.txt",
    "model": "./MobileNetSSD_deploy/MobileNetSSD_deploy.caffemodel",
    "confidence": 0.5
})
net = cv2.dnn.readNetFromCaffe(args.prototxt, args.model)

In [42]:
cap = cv2.VideoCapture("../video_input.mp4")

In [43]:
# look over the frames from the video
while(cap.isOpened()):
    ret, frame = cap.read()    
    frame = imutils.resize(frame, width=500)
    
    if W is None or H is None:
        (H, W) = frame.shape[:2]
    
    blob = cv2.dnn.blobFromImage(frame, 1.0, (W, H),
                                 (104.0, 177.0, 123.0))    
    net.setInput(blob)
    detections = net.forward()
    rects = []
    
    # loop over the detections
    for i in range(0, detections.shape[2]):
        # filter out weak detections by ensuring the predicted
        # probability is greater than a minimum threshold
        if detections[0, 0, i, 2] > args["confidence"]:
            # compute the (x, y)-coordinates of the bounding box for
            # the object, then update the bounding box rectangles list
            box = detections[0, 0, i, 3:7] * np.array([W, H, W, H])
            rects.append(box.astype("int"))
            # draw a bounding box surrounding the object so we can
            # visualize it
            (startX, startY, endX, endY) = box.astype("int")
            cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
    
    
    cv2.imshow("frame", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

AttributeError: 'NoneType' object has no attribute 'shape'

## Creating a trackable object

In [1]:
class TrackableObject:
    def __init__(self, objectID, centroid):
        # store the object ID and initialize a list of centroid location history
        self.objectID = objectID
        self.centroids = [centroid]
        
        # check if the object has already been counted or not
        self.counted = False